- VAE

![image](https://upload.wikimedia.org/wikipedia/commons/1/11/Reparameterized_Variational_Autoencoder.png)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from torchvision.transforms import ToTensor

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
train_images = datasets.MNIST(
    root= 'data',
    train= True,
    download= True,
    transform= ToTensor()
)

test_images = datasets.MNIST(
    root= 'data',
    train= True,
    download= True,
    transform= ToTensor()
)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 197MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 41.2MB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 52.3MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.46MB/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [3]:
# 하이퍼파라미터 준비
EPOCH = 10
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", DEVICE)

Using Device: cpu


In [4]:
train_loader = DataLoader(train_images, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_images, batch_size = BATCH_SIZE, shuffle = True)

In [5]:
class VAE(nn.Module):
    def __init__(self, latent_dim): #사용할 레이어 초기화
        super(VAE, self).__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(256, latent_dim)
        self.fc_var = nn.Linear(256, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 28 * 28),
            nn.Sigmoid(),
        )

    def encode(self, x):
        result = self.encoder(x)
        mu = self.fc_mu(result)
        var = self.fc_var(result)
        return mu, var

    def decode(self, z):
        result = self.decoder(z)
        return result

    def reparameterize(self, mu, var):
        std = torch.exp(var / 2)
        eps = torch.randn_like(std)
        return mu + (eps * std)

    def forward(self, x):
        x = self.flatten(x)
        mu, var = self.encode(x)
        z = self.reparameterize(mu, var)
        out = self.decode(z)
        return  out, mu, var

In [5]:
model = VAE(10).to(DEVICE)
print(model)